In [ ]:
import numpy as np
#from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.set_printoptions(precision=4)
np.random.seed(0)

In [ ]:
np.random.seed(0)
N = 100 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes
X = np.zeros((N*K,D))
y = np.zeros(N*K, dtype='uint8')
for j in range(K):
  ix = range(N*j,N*(j+1))
  r = np.linspace(0.0,1,N) # radius
  t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
  X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
  y[ix] = j

x_train = X.T
y_train = np.eye(len(set(y)))[y]
y_train = y_train.T

In [ ]:
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def mse(y_obs, y_pred):
    return np.mean(np.power(y_obs - y_pred, 2))

def mse_prime(y_obs, y_pred):
    return 2*(y_pred - y_obs) / y_obs.size

def softmax(x):
    tmp = np.exp(x)
    return tmp / np.sum(tmp, axis=0, keepdims=True)

def softmax_prime(x):                     # X(n,m) : matrix of softmax probabilities, #n classes, #m samples 
    i = np.eye(x.shape[0])                # Identity matrix with n dimensions
    tmp1 = np.einsum('ij,ik->jik', x, i)  # Matrix of diagolized softmax values (per m)
    tmp2 = np.einsum('ij,kj->jik', x, x)  # Matrix of cross product of softmax values (per m)
    return tmp1 - tmp2 

def centropy(x, y, eps=1e-8):
    x = x.clip(min=eps, max=None)
    logp = np.where(y==1, -np.log(x), 0)
    return logp.sum(axis=0)

def centropy_prime(x, y, eps=1e-8):
    x = x.clip(min=eps, max=None)
    return np.where(y==1, -1/x, 0)

def relu(x):
    return np.maximum(0, x)

def relu_prime(x):
    return (x > 0) * 1


In [ ]:
class Layer:
    def __init__(self) -> None:
        self.params = {}
        self.grads = {}
           
    def forward(self, input_data):
        raise NotImplementedError
    
    def backward(self, grad, alpha):
        raise NotImplementedError
        
class ZLayer(Layer):
    def __init__(self, inSize, outSize):
        super().__init__()
        np.random.seed(0)
        self.params['W'] = 0.01 * np.random.randn(outSize, inSize)
        self.params['B'] = 0.0 * np.random.randn(outSize, 1)
       
    def forward(self, inData):
        self.input = inData
        self.output = np.dot(self.params['W'], self.input) + self.params['B']
        return self.output
        
    def backward(self, outGrad, alpha=1e-0):
        inGrad = np.dot(self.params['W'].T, outGrad)
        self.grads['W'] = np.dot(outGrad, self.input.T)
        self.grads['B'] = np.sum(outGrad, axis=1, keepdims=True)
        return inGrad        
    
class ALayer(Layer):
    def __init__(self, g, gPrime):
        super().__init__()
        self.g = g
        self.gPrime = gPrime

    def forward(self, inData):
        self.input = inData
        self.output = self.g(self.input)
        return self.output

    def backward(self, outGrad):
        return self.gPrime(self.input) * outGrad

class SLayer(ALayer):
    def backward(self, outGrad):
        return np.einsum('jik,kj->ij', self.gPrime(self.output), outGrad)
    
class LLayer(Layer):
    def __init__(self, g, gPrime, eps=1e-8):
        self.g = g
        self.gPrime = gPrime
        self.eps = eps

    def forward(self, pred, obs):
        self.input = pred
        self.output = self.g(pred, obs, self.eps)
        return self.output

    def backward(self, pred, obs):
        return self.gPrime(pred, obs, self.eps)    
    
class Optimiser():
    def step(self, layers):
        raise NotImplementedError
        
class SGD(Optimiser):
    def __init__(self, alpha=1e-2):
        self.alpha = alpha
    
    def step(self, params, grads):
        for name, param in params.items():
            param -= self.alpha * grads[name]
            

In [ ]:
class Network:
    def __init__(self, loss = LLayer(centropy, centropy_prime), optimiser = SGD(alpha=1)):
        self.layers = []
        self.loss = loss
        self.optimiser = optimiser
        
    def add(self, layer):
        self.layers.append(layer)

    def forward(self, output):
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self, grad):
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
        return grad
    
    def update(self):
        for layer in self.layers:
            self.optimiser.step(layer.params, layer.grads)
        
    def accuracy(self, x, y):
        W = self.layers[0].params['W']
        B = self.layers[0].params['B']
        scores = np.dot(W, x) + B
        predicted_class = np.argmax(scores, axis=0)
        return np.mean(predicted_class == y)
    
    def boundary_plot(self, x):
        W = self.layers[0].params['W']
        B = self.layers[0].params['B']
        h = 0.02
        x_min, x_max = x[0, :].min() - 1, x[0, :].max() + 1
        y_min, y_max = x[1, :].min() - 1, x[1, :].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        Z = np.dot(np.c_[xx.ravel(), yy.ravel()], W.T) + B.T
        Z = np.argmax(Z, axis=1)
        Z = Z.reshape(xx.shape)
        fig = plt.figure()
        plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
        plt.scatter(x[0, :], x[1, :], c=y, s=40, cmap=plt.cm.Spectral)
        plt.xlim(xx.min(), xx.max())
        plt.ylim(yy.min(), yy.max())
    
    def fit(self, x_train, y_train, epochs, learning_rate, verbose=1):
        for i in range(epochs):
            y_pred = self.forward(x_train)

            epoch_loss = np.sum(self.loss.forward(y_pred, y_train)) / y_pred.shape[1]
            
            grad = self.loss.backward(y_pred, y_train) / y_pred.shape[1]            
            grad = self.backward(grad)
        
            self.update()
        
            if verbose > 1 or (verbose == 1 and i % (epochs/10) == 0):
                print('epoch %4.d/%d   error=%f' % (i+1, epochs, epoch_loss))

In [ ]:
nn = Network()
nn.add(ZLayer(2, 3))
nn.add(SLayer(softmax, softmax_prime))

nn.fit(x_train, y_train, 100, 1)

print('training accuracy: %.2f' % nn.accuracy(x_train, y))

nn.boundary_plot(x_train)


In [ ]:
nn = Network()
nn.add(ZLayer(2, 100))
nn.add(ALayer(relu, relu_prime))
nn.add(ZLayer(100, 3))
nn.add(SLayer(softmax, softmax_prime))

nn.fit(x_train, y_train, 100, 1, 1)

print('training accuracy: %.2f' % nn.accuracy(x_train, y))

In [ ]:
nn.boundary_plot(x_train)

In [ ]:
Z1 = ZLayer(2, 3)
Z1.forward(x_train)

A1 = SLayer(softmax, softmax_prime)
A1.forward(Z1.output)

L1 = LLayer(centropy, centropy_prime)
L1.forward(A1.output, y_train)

grad_ce = L1.backward(A1.output, y_train)
grad_sm = A1.backward(grad_ce)/grad_ce.shape[1]
grad_dW = Z1.backward(grad_sm)

print('probs.T \n', A1.output[:,0:5].T, A1.output.shape, '\n')
print('y_train.T \n', y_train[:,0:5].T, y_train.shape, '\n')

print('CE \n', L1.output[0:5], L1.output.shape, '\n')
print('loss \n', f"{np.sum(L1.output)/L1.output.shape[0] : .6f} \n")

print('grad_ce.T \n', grad_ce[:,0:5].T, grad_ce.shape, '\n')
print('grad_sm.T \n', grad_sm[:,0:5].T, grad_sm.shape, '\n')

print('dW.T \n', Z1.dW.T, Z1.dW.shape, '\n')
print('dB.T \n', Z1.dB.T, Z1.dB.shape, '\n')
print('W.T \n', Z1.W.T, Z1.W.shape, '\n')
print('B.T \n', Z1.B.T, Z1.B.shape, '\n')

In [ ]:
np.set_printoptions(precision=4)
np.random.seed(0)

x_train = np.array([[0,0], [0,1], [1,0], [1,1]]).T
y_train = np.array([[0], [1], [1], [0]]).T

L1 = FCLayer(2, 3)
L1.forward(x_train)
#print(L1.W)
#print(L1.B)
print(L1.input)
print(L1.output)

A1 = ActivationLayer(tanh, tanh_prime)
A1.forward(L1.output)
print(A1.output)

L2 = FCLayer(3, 1)
L2.forward(A1.output)

A2 = ActivationLayer(tanh, tanh_prime)
A2.forward(L2.output)
print(A2.output)



In [ ]:
import numpy as np
np.set_printoptions(precision=4)
#%matplotlib inline

# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

In [ ]:
class Network_:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate, verbose=False):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in tqdm(range(epochs)):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            if verbose:
                print('epoch %d/%d   error=%f' % (i+1, epochs, err))